In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

In [ ]:
#Code without chunks
# import csv

# df = pd.read_csv("test.csv")
# predict_summary=[]
# df = df.head(250)

# with open ("result.csv",'w',encoding="utf-8") as csvfile_out:
#   writer = csv.writer(csvfile_out)
#   for article in df["article"]:
#     inputs = tokenizer(article, return_tensors="pt", padding=True, max_length=1024, truncation=True)
#     outputs = model.generate(**inputs, max_new_tokens=10000)
#     summary = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     writer.writerow(summary)

In [18]:
import csv

def generate_output(input_text):
    max_chunk_length = 500
    input_chunks = [input_text[i:i + max_chunk_length] for i in range(0, len(input_text), max_chunk_length)]
    output_chunks = []

    for chunk in input_chunks:
        inputs = tokenizer(chunk, return_tensors="pt")
        outputs = model.generate(**inputs, max_length=10000)
        decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        output_chunks.append(decoded_output[0])

    return "".join(output_chunks)


input_csv_file = "test.csv"
output_csv_file = "result.csv"

with open(input_csv_file, "r", encoding="utf-8") as csvfile, open(output_csv_file, "w", newline="", encoding="utf-8") as csvfile_out:
    reader = csv.reader(csvfile)
    writer = csv.writer(csvfile_out)

    for row in reader:
        input_text = row[0]
        output_text = generate_output(input_text)
        writer.writerow([output_text])

print("Outputs generated and saved to output.csv.")

In [31]:
import pandas as pd
from evaluate import load

bertscore = load("bertscore")

df1 = pd.read_csv("result.csv")
df2 = pd.read_csv("book.csv")

df1.columns
df2.columns

Index(['Actual'], dtype='object')

In [37]:
predictions = df2['Actual'].tolist()
references = df1['Predicted'].tolist()

results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

precision = results["precision"]
recall = results["recall"]
f1 = results["f1"]

output_df = pd.DataFrame({"Actual": references, "Predicted": predictions, "Precision": precision, "Recall": recall, "F1": f1})

output_df.to_csv("bert_scores_output.csv", index=False)

final_precision = sum(precision) / len(precision)
final_recall = sum(recall) / len(recall)
final_f1 = sum(f1) / len(f1)

print("Final Cumulative BERT Scores:")
print("Precision:", final_precision)
print("Recall:", final_recall)
print("F1 Score:", final_f1)


Final Cumulative BERT Scores:
Precision: 0.7826485204696655
Recall: 0.7591246398289998
F1 Score: 0.7704060769081116


In [40]:
rouge = load('rouge')


references = df2["Actual"].tolist()
predictions = df1["Predicted"].tolist()

results = rouge.compute(predictions=predictions, references=references)

output_df = pd.DataFrame({"Actual": references, "Predicted": predictions, "ROUGE-1": results['rouge1'], "ROUGE-2": results['rouge2'], "ROUGE-L": results['rougeL'], "ROUGE-Lsum": results['rougeLsum']})

output_df.to_csv("rouge_scores_output.csv", index=False)

print("ROUGE Scores:")
print("ROUGE-1:", results['rouge1'])
print("ROUGE-2:", results['rouge2'])
print("ROUGE-L:", results['rougeL'])
print("ROUGE-Lsum:", results['rougeLsum'])

ROUGE Scores:
ROUGE-1: 0.24341470377226568
ROUGE-2: 0.06397838853002916
ROUGE-L: 0.15687973303448793
ROUGE-Lsum: 0.15706324397358995


In [44]:
from nltk.translate.bleu_score import sentence_bleu

def compute_bleu_score(prediction, references):
    return sentence_bleu(references, prediction)


references = df2["Actual"].tolist()
predictions = df1["Predicted"].tolist()

bleu_scores = [compute_bleu_score(prediction, references) for prediction, references in zip(predictions, references_list)]

output_df = pd.DataFrame({"Actual": df2["Actual"], "Predicted": predictions, "BLEU Score": bleu_scores})

output_df.to_csv("bleu_scores_output.csv", index=False)

average_bleu_score = sum(bleu_scores) / len(bleu_scores)

print("Average BLEU Score:", average_bleu_score)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU Score: 1.013431547498706e-231


In [3]:
import csv
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

def generate_output(input_text):
    max_chunk_length = 500
    input_chunks = [input_text[i:i + max_chunk_length] for i in range(0, len(input_text), max_chunk_length)]
    output_chunks = []

    for chunk in input_chunks:
        inputs = tokenizer(chunk, return_tensors="pt", max_length=1000, truncation=True)
        outputs = model.generate(**inputs, max_length=1000)
        decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        output_chunks.append(decoded_output[0])

    return "".join(output_chunks)

input_csv_file = "test.csv"
output_csv_file = "result.csv"

with open(input_csv_file, "r", encoding="utf-8") as csvfile, open(output_csv_file, "w", newline="", encoding="utf-8") as csvfile_out:
    reader = csv.reader(csvfile)
    writer = csv.writer(csvfile_out)

    for row in reader:
        input_text = row[0]
        output_text = generate_output(input_text)
        writer.writerow([output_text])

print("Outputs generated and saved to output.csv.")


Outputs generated and saved to output.csv.


In [15]:
import pandas as pd
from evaluate import load

bertscore = load("bertscore")

df1 = pd.read_csv("result.csv")
df2 = pd.read_csv("book.csv")

df1.columns


Index(['Predicted'], dtype='object')

In [16]:
references = df2['Actual'].tolist()
predictions = df1['Predicted'].tolist()

results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

precision = results["precision"]
recall = results["recall"]
f1 = results["f1"]

output_df = pd.DataFrame({"Actual": references, "Predicted": predictions, "Precision": precision, "Recall": recall, "F1": f1})

output_df.to_csv("bert_scores_output(BART).csv", index=False)

final_precision = sum(precision) / len(precision)
final_recall = sum(recall) / len(recall)
final_f1 = sum(f1) / len(f1)

print("Final Cumulative BERT Scores:")
print("Precision:", final_precision)
print("Recall:", final_recall)
print("F1 Score:", final_f1)


Final Cumulative BERT Scores:
Precision: 0.720064192612966
Recall: 0.8465023215611776
F1 Score: 0.7778871774673461


In [17]:
rouge = load('rouge')


references = df2["Actual"].tolist()
predictions = df1["Predicted"].tolist()

results = rouge.compute(predictions=predictions, references=references)

output_df = pd.DataFrame({"Actual": references, "Predicted": predictions, "ROUGE-1": results['rouge1'], "ROUGE-2": results['rouge2'], "ROUGE-L": results['rougeL'], "ROUGE-Lsum": results['rougeLsum']})

output_df.to_csv("rouge_scores_output(BART).csv", index=False)

print("ROUGE Scores:")
print("ROUGE-1:", results['rouge1'])
print("ROUGE-2:", results['rouge2'])
print("ROUGE-L:", results['rougeL'])
print("ROUGE-Lsum:", results['rougeLsum'])

ROUGE Scores:
ROUGE-1: 0.1929095923442734
ROUGE-2: 0.09380601385488575
ROUGE-L: 0.12922401690970325
ROUGE-Lsum: 0.1289632592433017


In [23]:
from nltk.translate.bleu_score import sentence_bleu

def compute_bleu_score(prediction, references):
    return sentence_bleu(references, prediction)

references_list = df2["Actual"].apply(lambda x: [reference.strip() for reference in x.split(',')]).tolist()
predictions = df1["Predicted"].tolist()

bleu_scores = [compute_bleu_score(prediction, references) for prediction, references in zip(predictions, references_list)]
output_df = pd.DataFrame({"Actual": df2["Actual"], "Predicted": predictions, "BLEU Score": bleu_scores})

output_df.to_csv("bleu_scores_output(BART).csv", index=False)

average_bleu_score = sum(bleu_scores) / len(bleu_scores)

print("Average BLEU Score:", average_bleu_score)

Average BLEU Score: 0.11331524389192421
